Convert this talk to a HTML preserntation via  
`jupyter nbconvert Jupyter\ Slides.ipynb --to slides --post serve`

https://medium.com/@mjspeck/presenting-code-using-jupyter-notebook-slides-a8a3c3b59d67

![title](title.jpg)

# Introduction

- Some text bruh

In [2]:
print('a fragment')

a fragment


In [4]:
print('a notes')

a notes


In [1]:
print('and back to slides')

and back to slides


# References

   - Molnar, Christoph. "Interpretable machine learning. A Guide for Making Black Box Models Explainable", 2019. https://christophm.github.io/interpretable-ml-book/.
   - Scholbeck CA, Molnar C, Heumann C, Bischl B, Casalicchio G (2019). Sampling, Intervention, Prediction, Aggregation: A Generalized Framework for Model Agnostic Interpretations. https://arxiv.org/abs/1904.03959
   - Doshi-Velez, Finale, and Been Kim. “Towards a rigorous science of interpretable machine learning,” no. Ml: 1–13. http://arxiv.org/abs/1702.08608 (2017).
   - Ribeiro, Marco Tulio, Sameer Singh, and Carlos Guestrin. "Model-agnostic interpretability of machine learning." https://arxiv.org/pdf/1606.05386 (2016).
   - Biggio, Battista, and Fabio Roli. "Wild patterns: Ten years after the rise of adversarial machine learning." Pattern Recognition 84 (2018): 317-331. https://arxiv.org/pdf/1712.03141